In [ ]:
import pydicom
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10,mnist
import numpy as np
import argparse
import cv2
import os
from tensorflow.keras import layers # just randommmmm
import matplotlib.pyplot as plt

In [ ]:
def load_dcm(filepath):
    return pydicom.read_file(filepath)
def rescale_dcm_to_hounsfield_img(dcm):
    slope, intercept = get_rescale_slope_intercept(dcm)
    img = dcm.pixel_array
    img = np.multiply(img, slope) + intercept
    return img.astype('float32')
def get_rescale_slope_intercept(dcm):
    slope = int(dcm.RescaleSlope)
    intercept = int(dcm.RescaleIntercept)
    return (slope, intercept)
def window_img(img, window_center, window_width):
    windowed_img = np.copy(img)
    px_max = window_center + (window_width // 2)
    px_min = window_center - (window_width // 2)
    windowed_img[windowed_img > px_max] = px_max
    windowed_img[windowed_img < px_min] = px_min
    return windowed_img
def plot_img(img):
    plt.figure(figsize=(10,6))
    plt.imshow(img, cmap=plt.cm.bone)

In [ ]:
class GAN:

    def build_generator(dim, depth, channels=1, inputDim=100,
        outputDim=512):
        model = Sequential()
        
        inputShape = (dim, dim, depth)
        chanDim = -1
        model.add(Dense(input_dim=inputDim, units=outputDim))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dense(dim * dim * depth))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Reshape(inputShape))
        model.add(Conv2DTranspose(32, (5, 5), strides=(2, 2),padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2DTranspose(channels, (5, 5), strides=(2, 2),padding="same"))
        model.add(Activation("tanh"))
       # model.add(layers.Dense((), activation="tanh"))

        return model
    def build_discriminator(width, height, depth, alpha=0.2):
        model = Sequential()
    #    print(height,width,depth)

        inputShape = (height, width, depth)
        model.add(Conv2D(32, (5, 5), padding="same", strides=(2, 2),
        input_shape=inputShape))
        model.add(LeakyReLU(alpha=alpha))
        model.add(Conv2D(64, (5, 5), padding="same", strides=(2, 2)))

        model.add(LeakyReLU(alpha=alpha))
        model.add(Flatten())
        print(model.output_shape)
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=alpha))
        model.add(Dense(1))
        model.add(Activation("sigmoid"))
        return model

In [ ]:
#parameter
NUM_EPOCHS = 50
BATCH_SIZE = 5
INIT_LR = 2e-4
OUTPUT_PATH = '.'

In [ ]:
dcm = load_dcm('../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_test/ID_000cef503.dcm')
img = rescale_dcm_to_hounsfield_img(dcm)
bone_window = window_img(img, 40, 80)
plot_img(bone_window)

In [ ]:
i = 0
imageList = list()
for image in  os.listdir('../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train'):
    if(i<=100):
        imageList.append(image)
        i+=1

In [ ]:
trainX = list()
testX = list()
i = 0
for e in imageList:
    dcm = load_dcm('../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train/'+e)
    img = rescale_dcm_to_hounsfield_img(dcm)
    bone_window = window_img(img, 40, 80)
    if(i<=80):
        trainX.append(bone_window)
    else:
        testX.append(bone_window)
    i+=1
trainX = np.array(trainX)
testX = np.array(testX)

In [ ]:
plt.imshow(testX[3])

In [ ]:
gen = GAN.build_generator(128, 64, channels=1)
#dis = GAN.build_discriminator(32, 32, 1)
dis = GAN.build_discriminator(512, 512, 1)
dis.trainable = False

In [ ]:
optimizer = Adam(lr=INIT_LR, beta_1=0.5, decay=INIT_LR / NUM_EPOCHS)
dis.compile(loss="binary_crossentropy", optimizer=optimizer)

In [ ]:
ganInput = Input(shape=(100,))
ganOutput = dis(gen(ganInput))
gan = Model(ganInput, ganOutput)
ganOpt = Adam(lr=INIT_LR, beta_1=0.5, decay=INIT_LR / NUM_EPOCHS)
gan.compile(loss="binary_crossentropy", optimizer=optimizer)

In [ ]:
benchmarkNoise  = np.random.uniform(-1, 1, size=(256, 100))

In [ ]:
!pip install imutils

In [ ]:
from sklearn.utils import shuffle
from imutils import build_montages
#print(trainX.shape,testX.shape)
result = list()
trainImages = np.concatenate([trainX, testX])

trainImages = np.expand_dims(trainImages, axis=-1)
trainImages = (trainImages.astype("float") - 127.5) / 127.5

for epoch in range(0, NUM_EPOCHS):
   
    batchesPerEpoch = int(trainImages.shape[0] / BATCH_SIZE)
    print(batchesPerEpoch,'batch')
    
    for i in range(0, batchesPerEpoch):
        p = None
      
        imageBatch = trainImages[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
        noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))
      #  print(noise.shape)
        genImages = gen.predict(noise, verbose=0)
        imageBatch = imageBatch.reshape((BATCH_SIZE,512,512,1))
        #print('batch gen',imageBatch.shape,genImages.shape)
        X = np.concatenate((imageBatch, genImages))
        del genImages,imageBatch
        y = ([1] * BATCH_SIZE) + ([0] * BATCH_SIZE)
        y = np.reshape(y, (-1,))
        (X, y) = shuffle(X, y)

        discLoss = dis.train_on_batch(X, y)
        noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
        fakeLabels = [1] * BATCH_SIZE
        fakeLabels = np.reshape(fakeLabels, (-1,))
        ganLoss = gan.train_on_batch(noise, fakeLabels)
        if i == batchesPerEpoch - 1:
            p = [OUTPUT_PATH, "epoch_{}_output.png".format(str(epoch + 1).zfill(4))]
        if p is not None:
            print('output path ',p)
            print('result ',epoch + 1, i,discLoss, ganLoss)
    
            images = gen.predict(benchmarkNoise)
            images = ((images * 127.5) + 127.5).astype("uint8")
            images = np.repeat(images, 3, axis=-1)
            result.append(images)
            vis = build_montages(images, (32, 32), (16, 16))[0]
            p = os.path.sep.join(p)
            cv2.imwrite(p, vis)
        if p is None:
            print('doing',discLoss,ganLoss,epoch+1,i)
